# Preprocesamiento

Este cuaderno recoge las operaciones de preprocesamiento y preparación de los datasets para su posterior uso en el dashboard.
Los conjuntos de datos utilizados han sido directamente extraídos de tres fuentes:
- Universidata. https://www.universidata.es/search/type/dataset
- Instituto Nacional de Estadística. https://www.ine.es/
- Informes "Anuario estadístico. Las cifras de la educación en España" del Ministerio de Educación y Deportes. https://www.educacionfpydeportes.gob.es/servicios-al-ciudadano/estadisticas/indicadores/cifras-educacion-espana.html

In [13]:
import pandas as pd
import zipfile
import os
import re
import numpy as np
import unicodedata

In [14]:
if not os.path.exists("smart/"):
    os.makedirs("smart/")
if not os.path.exists("raw/"):
    os.makedirs("raw/")

smart = "smart/"

# Matriculas

In [15]:
origen = 'comprimido/'
destino = 'raw/'
with zipfile.ZipFile(origen+"univaciada_1.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)
with zipfile.ZipFile(origen+"univaciada_2.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)
with zipfile.ZipFile(origen+"univaciada_3.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)

In [16]:
#Preprocesamiento matrículas
ficheros = os.listdir(destino)
#Solo cargamos las columnas necesarias
columnas_interes = ["curso_academico", 'des_municipio_residencia',"cod_tipo_estudio", "cod_municipio_residencia",  "lat_municipio_residencia", "lon_municipio_residencia"]
dataframes = []
#Limpieza de datos, nos quedamos solo con estudiantes de Grado españoles. Añadimos una columna con la universidad del estudiante.
for fichero in ficheros:
    try:
        df = pd.read_csv(destino+"/"+fichero, usecols=columnas_interes, low_memory=False)
        df = df[df["cod_tipo_estudio"] == 'G']  
        df = df[df['des_municipio_residencia'] != 'MUNICIPIO EXTRANJERO PERTENECIENTE A LA U.E.']
        df = df[df['des_municipio_residencia'] != 'MUNICIPIO EXTRANJERO NO PERTENECIENTE A LA U.E.']
        df = df.drop(columns=["des_municipio_residencia"])
        df = df.dropna()
        df['cod_municipio_residencia'] = df['cod_municipio_residencia'].astype('int')
        nombre = fichero.split('/')[-1]
        universidad = re.match(r'^([^-]+)', nombre).group(1)
        df['universidad'] = universidad
        dataframes.append(df)  
    except Exception as e:
        print(f"Error al cargar el archivo desde {fichero}: {e}")

df_matriculas = pd.concat(dataframes, ignore_index=True)
df_matriculas['count'] = 1
df_grouped = df_matriculas.groupby(['lat_municipio_residencia', 'lon_municipio_residencia', "cod_municipio_residencia",'universidad', 'curso_academico'], as_index=False).agg({
    'count': 'sum'
})

min_count = df_grouped['count'].min()
max_count = df_grouped['count'].max()
df_grouped['log_count'] = df_grouped['count'].apply(lambda x: max(1, x)).apply(np.log)
df_grouped

Error al cargar el archivo desde colegios.csv: Usecols do not match columns, columns expected but not found: ['cod_tipo_estudio', 'cod_municipio_residencia', 'curso_academico', 'lat_municipio_residencia', 'lon_municipio_residencia', 'des_municipio_residencia']
Error al cargar el archivo desde datosSimulador.csv: Usecols do not match columns, columns expected but not found: ['cod_tipo_estudio', 'cod_municipio_residencia', 'lat_municipio_residencia', 'lon_municipio_residencia', 'des_municipio_residencia']
Error al cargar el archivo desde prediccionesSimulador.csv: Usecols do not match columns, columns expected but not found: ['cod_tipo_estudio', 'cod_municipio_residencia', 'lat_municipio_residencia', 'lon_municipio_residencia', 'des_municipio_residencia']


,lat_municipio_residencia,lon_municipio_residencia,cod_municipio_residencia,universidad,curso_academico,count,log_count
0,27.75379,-18.003640,38013,uam,2017-18,1,0.000000
1,27.75379,-18.003640,38013,ucm,2017-18,5,1.609438
2,27.75379,-18.003640,38013,ucm,2018-19,4,1.386294
3,27.75379,-18.003640,38013,ucm,2019-20,4,1.386294
4,27.75379,-18.003640,38013,ucm,2020-21,4,1.386294
...,...,...,...,...,...,...,...
46021,43.73261,-7.674875,27064,uam,2022-23,1,0.000000
46022,43.73261,-7.674875,27064,uc3m,2020-21,1,0.000000
46023,43.73261,-7.674875,27064,uc3m,2021-22,1,0.000000
46024,43.73261,-7.674875,27064,uc3m,2022-23,1,0.000000


In [17]:
#También necesitamos los datos del INE para saber la densidad de los municipios
with zipfile.ZipFile(origen+"INe.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)
    
ine = pd.read_csv(destino+"INe.csv", sep=";", low_memory=False)
#Nos quedamos con la población total del 2017 en adelante
ine = ine[ine["Sexo"] == "Total"]
ine = ine[ine["Periodo"] >= 2017]
ine[['cod_municipio_residencia', 'municipio']] = ine['Municipios'].str.split(' ', n=1, expand=True)
ine['cod_municipio_residencia'] = ine['cod_municipio_residencia'].astype(int)
ine['curso_academico'] = ine['Periodo'].astype(int).apply(lambda x: f"{x}-{(x + 1) % 100:02d}")
ine = ine.dropna()
ine['Total'] = ine['Total'].str.replace('.', '', regex=False).astype(int)
ine['Total'] = ine['Total'].astype('int')
ine['poblacion'] = ine['Total'].apply(
    lambda x: 'Muy densa' if x > 500000 else
              'Densa' if x > 100000 else
              'Moderadamente densa' if x > 25000 else
              'Poco densa' if x > 5000 else
              'Muy poco densa'
)
ine = ine.drop(columns=['Municipios',"Sexo","Periodo"])
ine

,Total,cod_municipio_residencia,municipio,curso_academico,poblacion
0,74,44001,Ababuj,2024-25,Muy poco densa
1,70,44001,Ababuj,2023-24,Muy poco densa
2,72,44001,Ababuj,2022-23,Muy poco densa
3,76,44001,Ababuj,2021-22,Muy poco densa
4,77,44001,Ababuj,2020-21,Muy poco densa
...,...,...,...,...,...
707922,2895,4103,Zurgena,2021-22,Muy poco densa
707923,2956,4103,Zurgena,2020-21,Muy poco densa
707924,3005,4103,Zurgena,2019-20,Muy poco densa
707925,2893,4103,Zurgena,2018-19,Muy poco densa


In [18]:
#Juntamos los conjuntos
df_merged = df_grouped.merge(ine, on=['cod_municipio_residencia', 'curso_academico'], how='left')
df_merged.to_csv(smart+'matricula.csv', index=False)

In [19]:
#Para el gráfico 2 necesitamos el porcentaje con respecto a la población total
df_g2 = df_merged[['universidad','curso_academico','count','Total','poblacion']]
df_final = df_g2.groupby(['universidad', 'curso_academico', 'poblacion'], as_index=False).agg({'count': 'sum', 'Total': 'sum'})

#Calculamos los porcentajes
poblacion_todos = ine.groupby(['curso_academico', 'poblacion'], as_index=False).agg({'Total': 'sum'})
poblacion_todos=poblacion_todos.rename(columns={'Total': 'total_todos'})
df_final2 = df_final.merge(poblacion_todos, on=['poblacion', 'curso_academico'], how='left')
df_final2['porcentaje_todos'] = (df_final2['count'] / df_final2['total_todos'])
df_final2.to_csv(smart+'matriculaG2.csv', index=False)

# Acceso

In [20]:
if not os.path.exists(destino+"acceso/"):
    os.makedirs(destino+"acceso/")
with zipfile.ZipFile(origen+"acceso.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino+"acceso/")

In [21]:
ficheros = os.listdir(destino+"acceso/")
#Solo cargamos las columnas necesarias
columnas_interes = ["curso_academico", 'des_titulacion',"des_provincia_centro_sec","cod_municipio_centro_sec","nota_admision"]
dataframes = []

#Limpieza de datos, nos quedamos solo con estudiantes de Grado españoles
for fichero in ficheros:
    try:
        df = pd.read_csv(destino+"acceso/"+fichero, usecols=columnas_interes, low_memory=False)   
        df = df.dropna()
        dataframes.append(df)  
    except Exception as e:
        print(f"Error al cargar el archivo desde {fichero}: {e}")

df_acceso = pd.concat(dataframes, ignore_index=True)

#Unificamos la descripción de las titulaciones
#Letras en minúscula
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.lower()
df_acceso['cod_municipio_centro_sec'] = df_acceso['cod_municipio_centro_sec'].astype('int')
df_acceso['nota_admision'] = df_acceso['nota_admision'].str.replace(',', '.').astype(float)

#Quitamos las tildes para normalizar
def quitar_tildes(texto):
    if isinstance(texto, str):  
        texto_normalizado = unicodedata.normalize('NFD', texto)
        texto_sin_tildes = ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')
        return texto_sin_tildes
    return texto  
#Quitamos los dobles grados porque ya aparecen los individuales por separado
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('doble', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('triple', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('/', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('programa de estudios conjuntos ', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('programa de estudios conjunto ', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('programa conjunto ', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('programa de estudio conjunto ', case=False, na=False)]

#Quitamos tildes para normalizar
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].apply(quitar_tildes)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.strip().str.lower()
#Quitamos el "por la Universidad..."
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r' por.*', '', regex=True)
#Normalizamos
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^graduado o graduada en ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^grado en ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^degree in ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^graduado/a en ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^maestro en ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^maestro de ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'o/a', 'o', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'grado abierto uc3m en', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r' de segovia', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'\(.*?\)', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.split(' centro').str[0]
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.split(' instituto').str[0]
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r' i$', '', regex=True)

df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.strip().str.lower()

In [22]:
ine = ine.rename(columns={'cod_municipio_residencia': 'cod_municipio_centro_sec'})
df_merged2 = df_acceso.merge(ine, on=['cod_municipio_centro_sec', 'curso_academico'], how='left')
notas = df_merged2.groupby(['curso_academico', 'des_provincia_centro_sec']).agg(
    nota_media=('nota_admision', 'mean'),
).reset_index()

#Modificamos para unificar con el JSON
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Valencia/València', 'Valencia')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Alicante/Alacant', 'Alicante')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Castellón/Castelló', 'Castellón')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Girona', 'Gerona')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Lleida', 'Lérida')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('A Coruña', 'La Coruña')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Ourense', 'Orense')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Guipúzcoa', 'Gipuzkoa')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Vizcaya', 'Bizkaia')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Illes Balears', 'Baleares')

#Notas medias para el mapa de la pestaña 2
notas.to_csv(smart+'medias.csv', index=False) 

#Calculamos el resto de estadísticas para el diagrama de caja y bigote
df_merged2 = df_merged2.dropna()
curso_academico_vals = df_merged2['curso_academico'].unique()
poblacion_vals = df_merged2['poblacion'].unique()
provincia_vals = df_merged2['des_provincia_centro_sec'].unique()

all_combinations = pd.MultiIndex.from_product(
    [curso_academico_vals, poblacion_vals, np.append(provincia_vals, 'Global')],
    names=['curso_academico', 'poblacion', 'des_provincia_centro_sec']
).to_frame(index=False)

provincia = df_merged2.groupby(['curso_academico', 'poblacion','des_provincia_centro_sec']).agg(
    nota_media=('nota_admision', 'mean'),
    nota_minima=('nota_admision', 'min'),
    nota_maxima=('nota_admision', 'max'),
    percentil_25=('nota_admision', lambda x: x.quantile(0.25)),
    percentil_50=('nota_admision', lambda x: x.quantile(0.50)),
    percentil_75=('nota_admision', lambda x: x.quantile(0.75))
).reset_index()

global_stats = df_merged2.groupby(['curso_academico', 'poblacion']).agg(
    nota_media=('nota_admision', 'mean'),
    nota_minima=('nota_admision', 'min'),
    nota_maxima=('nota_admision', 'max'),
    percentil_25=('nota_admision', lambda x: x.quantile(0.25)),
    percentil_50=('nota_admision', lambda x: x.quantile(0.50)),
    percentil_75=('nota_admision', lambda x: x.quantile(0.75))
).reset_index()

global_stats['des_provincia_centro_sec'] = 'Global'

stats = pd.concat([provincia, global_stats], ignore_index=True)
notas = pd.merge(all_combinations, stats, on=['curso_academico', 'poblacion', 'des_provincia_centro_sec'], how='left')

#Modificamos para unificar con el JSON
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Valencia/València', 'Valencia')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Alicante/Alacant', 'Alicante')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Castellón/Castelló', 'Castellón')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Girona', 'Gerona')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Lleida', 'Lérida')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('A Coruña', 'La Coruña')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Ourense', 'Orense')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Guipúzcoa', 'Gipuzkoa')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Vizcaya', 'Bizkaia')
notas['des_provincia_centro_sec'] = notas['des_provincia_centro_sec'].replace('Illes Balears', 'Baleares')

notas.to_csv(smart+'notas.csv', index=False)  

In [23]:
#Top 5 titulaciones para la tabla de la pestaña 2
grouped = (
    df_merged2.groupby(['curso_academico', 'poblacion', 'des_titulacion'])
    .size()
    .reset_index(name='count')
)

grouped = grouped.sort_values(by=['curso_academico', 'poblacion', 'count'], ascending=[True, True, False])

grouped['orden'] = grouped.groupby(['curso_academico', 'poblacion'])['count'].rank(ascending=False, method='first').astype(int)

top5 = (
    grouped[grouped['orden'] <= 5]
    .sort_values(by=['curso_academico', 'poblacion', 'orden'])
    .reset_index(drop=True)
)
top5['des_titulacion'] = top5['des_titulacion'].replace('derecho', 'Derecho')
top5['des_titulacion'] = top5['des_titulacion'].replace('administracion y direccion de empresas', 'Administración y Dirección de Empresas')
top5['des_titulacion'] = top5['des_titulacion'].replace('educacion primaria', 'Educación Primaria')
top5['des_titulacion'] = top5['des_titulacion'].replace('periodismo', 'Periodismo')
top5['des_titulacion'] = top5['des_titulacion'].replace('educacion infantil', 'Educación Infantil')
top5['des_titulacion'] = top5['des_titulacion'].replace('economia', 'Economía')
top5['des_titulacion'] = top5['des_titulacion'].replace('psicologia', 'Psicología')
top5['des_titulacion'] = top5['des_titulacion'].replace('enfermeria', 'Enfermería')
top5['des_titulacion'] = top5['des_titulacion'].replace('comercio', 'Comercio')
top5['des_titulacion'] = top5['des_titulacion'].replace('medicina', 'Medicina')

top5.to_csv(smart+'top5.csv', index=False) 

# Colegios

In [25]:
#El conjunto que cargamos lo hemos construido manualmente a través de los informes "Anuario estadístico. Las cifras de la educación en España"
#Disponibles en https://www.educacionfpydeportes.gob.es/servicios-al-ciudadano/estadisticas/indicadores/cifras-educacion-espana.html
#No requiere preprocesamiento
colegios = pd.read_csv(destino+"colegios.csv",sep=";")

# Población jóven

In [7]:
#Datos extraidos del INe. Descomprimimos y unimos el dataset
with zipfile.ZipFile(origen+"poblacionRangoEdad1.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)
with zipfile.ZipFile(origen+"poblacionRangoEdad2.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)
df1 = pd.read_csv(destino+"poblacionRangoEdad1.csv", low_memory=False, sep=";")
df2 = pd.read_csv(destino+"poblacionRangoEdad2.csv", low_memory=False, sep=";")
df_concatenado = pd.concat([df1, df2], ignore_index=True)
df_concatenado.to_csv(destino+"poblacionRangoEdad.csv", index=False)